## importing packages

In [ ]:
import tensorflow as tf
import numpy as np
from nltk.tokenize import RegexpTokenizer
from itertools import islice
import matplotlib.pyplot as plt
import multiprocessing
from gensim.models import Word2Vec

## importing data

In [ ]:
# load dataset
dataset = pd.read_csv('data.csv')

In [ ]:
del dataset['id'], dataset['qid1'], dataset['qid2']

## Preprocessing 

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')

sentence1,sentence2=[],[]
words=[]
label=[]

for _,x in dataset.iterrows():
    
        tr=map(str.lower,tokenizer.tokenize(x[0]))
        sentence1.append(tr)
        words.extend(tr)
    
        tr=map(str.lower,tokenizer.tokenize(x[1])) 
        sentence2.append(tr)
        words.extend(tr)
        label.append(x[2])
        

In [ ]:
sentence=sentence1+sentence2
len(sentence)

In [ ]:
len(sentence1),len(sentence2)

## creating word embeddings 

In [ ]:
cores = multiprocessing.cpu_count()
model = Word2Vec(sentence, min_count=1, size=200, sg=1, iter=2, negative=10, workers=cores)
model.save('./word2vec.model')

In [ ]:
# load trained word2vec model
model = Word2Vec.load('./word2vec.model')
vocab = model.wv.vocab.keys()
vocab_size = len(vocab)


## creating word to index and index to word dictionary

In [ ]:
# index to word dictionary
id_to_word = dict(enumerate(list(vocab) ))

# word to index dictionary
word_to_id = {v: k for k, v in id_to_word.items()}

In [ ]:
max_length = max([len(i) for i in sentence])

## padding to make seq of equal length

In [ ]:
for i in range(len(sentence1)):
    sentence1[i] = [word_to_id[j] for j in sentence1[i]] + [vocab_size] * (max_length - len(sentence1[i]))
    sentence2[i] = [word_to_id[j] for j in sentence2[i]] + [vocab_size] * (max_length - len(sentence2[i]))


In [ ]:
sentence1=np.array(sentence1)
sentence2=np.array(sentence2)

## word embeddings for seq mapped to their indices

In [ ]:
vec_x = np.zeros([vocab_size+1, 200])
for i, j in id_to_word.items():
    vec_x[i] = model[j]

vec_x[vocab_size]=np.ones(200)
print vec_x[vocab_size]

## splitting dataset into train,validation and test set randomly

In [ ]:
train_sent1=sentence1[:209991]
train_sent2=sentence2[:209991]
label_train=label[:209991]
label_test=label[209991:]
test_sent1=sentence1[209991:]
test_sent2=sentence2[209991:]



In [ ]:
val_sent1=test_sent1[89895:]
val_sent2=test_sent2[89895:]
test_sent1=test_sent1[:89895]
test_sent2=test_sent2[:89895]
label_val=label_test[89895:]
label_test=label_test[:89895]

## saving the preprocessed data

In [ ]:
np.savez('/preprocessed_data/train.npz', ques1=train_sent1, ques2=train_sent2, label=label_train)
np.savez('/preprocessed_data/valid.npz', ques1=val_sent1, ques2=val_sent2, label=label_val)
np.savez('/preprocessed_data/test.npz', ques1=test_sent1, ques2=test_sent2, label=label_test)
np.savez(''/preprocessed_data/embed.npz',embed=vec_x)